In [1]:
import shutil
import os

In [2]:
pwd = os.getcwd()
#pwd += "/rheo_bench/channel/"
pwd

'/qscratch/etpaler/rheo_benchmark_repository/benchmark_1__taylor_green_vortex_flow/2D_vortex'

# Generate LAMMPS Input Scripts

In [3]:
#parameter grids for varying sf
var_shift_grid = {"":"0","shift":"1"}
# var_shift_grid = {"shift":"1"}

var_kernel_grid = {"CRK0":"0","CRK1":"1","CRK2":"2","Quintic":"5"}
var_sf_grid = {"0.5":"2","0.25":"4","0.2":"5","0.1":"10","0.05":"20"}
var_hd_grid = {"3.5":"3p5"}
var_eta_grid = {"0.05":"p05"}
var_u0_grid = {"0.05":"p05"}

In [4]:
#Generate lammps files

out_filepath = "data"

filenames = []
for shift in var_shift_grid:
    for kernel in var_kernel_grid:
        for sf in var_sf_grid:
            for hd in var_hd_grid:
                for eta in var_eta_grid:
                    for u0 in var_u0_grid:
                        filename = var_shift_grid[shift]+"_"+var_kernel_grid[kernel]+"_"+var_sf_grid[sf]+"_"+var_hd_grid[hd]+"_"+var_eta_grid[eta]+"_"+var_u0_grid[u0]
                        file = shutil.copyfile("lammps_in.template",pwd+"/in."+filename+".lmp")
                        file_in = open(file,"rt")
                        file_text = file_in.read()

                        #Replace variables
                        file_text = file_text.replace("var_shift",shift)
                        file_text = file_text.replace("var_kernel",kernel)
                        file_text = file_text.replace("var_sf",sf)
                        file_text = file_text.replace("var_hd",hd)
                        file_text = file_text.replace("var_eta",eta)
                        file_text = file_text.replace("var_u0",u0)

                        file_text = file_text.replace("var_out_filepath",out_filepath)
                        file_text = file_text.replace("var_filename",filename)

                        file_out = open(file,"wt")
                        file_out.write(file_text)
                        
                        filenames.append(filename)
                        file_out.close()

In [ ]:
print(filenames)

['1_0_2_3p5_p05_p05', '1_0_4_3p5_p05_p05', '1_0_5_3p5_p05_p05', '1_0_10_3p5_p05_p05', '1_0_20_3p5_p05_p05', '1_1_2_3p5_p05_p05', '1_1_4_3p5_p05_p05', '1_1_5_3p5_p05_p05', '1_1_10_3p5_p05_p05', '1_1_20_3p5_p05_p05', '1_2_2_3p5_p05_p05', '1_2_4_3p5_p05_p05', '1_2_5_3p5_p05_p05', '1_2_10_3p5_p05_p05', '1_2_20_3p5_p05_p05', '1_5_2_3p5_p05_p05', '1_5_4_3p5_p05_p05', '1_5_5_3p5_p05_p05', '1_5_10_3p5_p05_p05', '1_5_20_3p5_p05_p05']


# Write the Solo Jobscript

In [ ]:
account="FY150028"
job_name="TG_vortex"
time="48:00:00"
partition="batch"
nodes="4"
ntasks_per_node="16"
run_command = "mpirun -np 16 /ascldap/users/etpaler/dev/rlammps_king_laplacian/build/lmp -in in."

In [ ]:
with open("submit.jobscript","w") as jobscript:
        #---shebang---#
        jobscript.write("#!/bin/bash\n\n")

        #---slurm commands---#
        jobscript.write("#SBATCH --account={}\n".format(account))
        jobscript.write("#SBATCH --job-name={}\n".format(job_name))
        jobscript.write("#SBATCH --time={}\n".format(time))
        jobscript.write("#SBATCH --partition={}\n".format(partition))
        jobscript.write("#SBATCH --nodes={}\n".format(nodes))
        jobscript.write("#SBATCH --ntasks-per-node={}\n\n".format(ntasks_per_node))
        
         #---unix commands---#
        jobscript.write("module load cmake/3.20.3 intel/18.0.0.128 openmpi-intel/4.1 mkl/18.0.0.128\n\n")
        
        for filename in filenames:
            jobscript.write(run_command+filename+".lmp \n")
            
        jobscript.write("\n")

# Write the WSL Jobscript

In [ ]:
run_command = "mpirun -np 4 ~/dev/rlammps_wall_debug/build/lmp -in lammps_in__"

with open("jobscript","w") as jobscript:
        #---shebang---#
        jobscript.write("#!/bin/bash\n\n")

        for filename in filenames:
            jobscript.write(run_command+filename+"\n")
            
        jobscript.write("\n")